# Using Mem0 Memory Store with Haystack Agents

[Mem0](https://mem0.ai/) provides a managed memory layer that allows agents to store and retrieve user-specific facts across conversations. In this notebook, we will:

1. Set up a `Mem0MemoryStore` and add memories about a user
2. Create a Haystack Agent that uses the memory store
3. Ask the Agent a personalized question and see how it leverages stored memories

## Install the required dependencies

We need `haystack-ai` for the core framework, `haystack-experimental` for the Agent with memory store support, and `mem0ai` for the Mem0 integration.

In [ ]:
!pip install haystack-ai haystack-experimental mem0ai

## Set up API keys

This notebook requires two API keys:
- **`OPENAI_API_KEY`**: Used by the [`OpenAIChatGenerator`](https://docs.haystack.deepset.ai/docs/openaichatgenerator) to power the Agent's LLM.
- **`MEM0_API_KEY`**: Used by `Mem0MemoryStore` to connect to the [Mem0 Platform](https://app.mem0.ai/). You can get a free API key by signing up.

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

if not os.environ.get("MEM0_API_KEY"):
    os.environ["MEM0_API_KEY"] = getpass("Enter your Mem0 API key: ")

## Create the Mem0 Memory Store

The `Mem0MemoryStore` connects to the Mem0 Platform using the `MEM0_API_KEY` environment variable. We'll add several facts about a user so the Agent can later use this context to give personalized answers.

In [ ]:
from haystack.dataclasses import ChatMessage
from haystack_experimental.memory_stores.mem0 import Mem0MemoryStore

memory_store = Mem0MemoryStore()

messages = [
    ChatMessage.from_user("I like to listen to Russian pop music"),
    ChatMessage.from_user("I liked cold spanish latte with oat milk"),
    ChatMessage.from_user("I live in Florence Italy and I love mountains"),
    ChatMessage.from_user(
        "I am a software engineer and I like building application in python. "
        "Most of my projects are related to NLP and LLM agents. "
        "I find it easier to use Haystack framework to build my projects."
    ),
    ChatMessage.from_user(
        "I work in a startup and I am the CEO of the company. "
        "I have a team of 10 people and we are building a platform "
        "for small businesses to manage their customers and sales."
    ),
]

memory_store.add_memories(user_id="agent_example", messages=messages)

## Create the Agent with Memory

Now we create a Haystack Agent that uses an `OpenAIChatGenerator` as its LLM and the `Mem0MemoryStore` as its memory.

When the Agent runs, it will:
1. Search the memory store using the user's message as a query
2. Inject relevant memories into the conversation as context
3. Generate a response that takes those memories into account
4. Save new messages back to the memory store automatically

In [ ]:
from haystack.components.generators.chat.openai import OpenAIChatGenerator
from haystack_experimental.components.agents.agent import Agent

chat_generator = OpenAIChatGenerator()

agent = Agent(
    chat_generator=chat_generator,
    memory_store=memory_store,
)

## Ask a Personalized Question

Let's ask the Agent a question that it can only answer well if it remembers the user's preferences. Based on the stored memories, the Agent knows the user is a Python developer who uses Haystack and works on NLP/LLM projects — so it should recommend Haystack for building the travel agent.

In [ ]:
result = agent.run(
    messages=[ChatMessage.from_user(
        "Based on what you know about me, which framework should I use to design an AI travel agent?"
    )],
    memory_store_kwargs={"user_id": "agent_example"},
)

print(result["messages"][-1].text)

## Clean Up

Since memories are stored in your Mem0 account, let's clean up the memories we created in this notebook to avoid leaving orphaned data.

In [ ]:
memory_store.delete_all_memories(user_id="agent_example")